In [1]:
%load_ext Cython
%time
from gensim.models import doc2vec 
import nltk 
import re 
import pandas as pd
import numpy as np


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [2]:
####load data and save as a txt####
raw_data=pd.read_csv("data.csv")
np.savetxt(r'review.txt', raw_data['review'], fmt='%s')

In [3]:
def split_sentence(sentence): 
    return re.split('\W+', sentence) 
class MyDocs(object): 
    def __iter__(self):
        for i, text in enumerate(open("review.txt")): 
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i]) 
# Train the doc2vec model cdef 
mydocs = MyDocs() 
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4) 
model.save('review.model') 

In [4]:
print model.most_similar(positive=["game", "great"], negative=["bad"], topn=3)

[('app', 0.7457967400550842), ('job', 0.6689680814743042), ('Love', 0.6604655981063843)]


In [25]:
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 
# Get the specific line of the review text file
def get_line(filename, n): 
    with open(filename) as f: 
        for i, e in enumerate(f): 
            if i == n: return e 
# Calculate the vector of input text according to our training model 
input_text = "great puzzle game" 
input_vec = model.infer_vector(split_sentence(input_text)) 

In [26]:
cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 

In [27]:
# Get the index of top n most similar review 
def argmaxn(l, n): 
    l_copy = list(l) 
    args = [] 
    for i in range(n): 
        arg = np.argmax(l_copy) 
        args.append(arg) 
        l_copy[arg] = -float('inf') 
        return args 

In [28]:
# Print the reviews with the highest cosine similarity values 
sim_ids = argmaxn(cossims_with_input, 3) 
for i in range(len(sim_ids)): 
    print "(Review-%s, Similarity:%.4f):" % (sim_ids[i] + 1, cossims_with_input[sim_ids[i]]),\
    get_line('review.txt', sim_ids[i]) 
    

(Review-6784, Similarity:0.8765): The game is very fun, but the amount of little bugs in the game bothers me; tricks wont register or multiplier decides to not go up. I love the game but just wish it would run smooth, especially after its been out this long



In [29]:
# Construct a data frame with the Appname and document-vectors of all archived reviews 
raw_data['Textvec'] = model.docvecs 
# Compute similarity scores between the input text and the hotels (defined here as the cosine similarity between 
# the document-vector of the input text and the sum/mean of the document-vectors of all the reviews of a hotel -- 
# no difference between sum and mean) 
app_cossims = raw_data.groupby('app').agg(lambda v: cossim(np.sum(v), input_vec))
# Print the reviews with the highest similarity scores 
sim_ids = argmaxn(app_cossims['Textvec'], 3) 
for i in range(len(sim_ids)): 
    app = app_cossims.index[sim_ids[i]] 
    print "App name is: %s" % app, ", and the similarity is equal to: %.4f," % app_cossims.Textvec[app]

App name is: Block! Hexa Puzzle , and the similarity is equal to: 0.7963,
